In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np

In [89]:
#data = pro.daily(ts_code=code, start_date='20210101', end_date='20211231')
data = pd.read_csv('.\TRD_Dalyr.csv')

In [90]:
data

,Stkcd,Trddt,Opnprc,Clsprc
0,600000,2020-01-02,12.47,12.47
1,600000,2020-01-03,12.57,12.60
2,600000,2020-01-06,12.52,12.46
3,600000,2020-01-07,12.51,12.50
4,600000,2020-01-08,12.41,12.32
...,...,...,...,...
11445,603986,2020-12-25,188.15,183.13
11446,603986,2020-12-28,183.01,184.25
11447,603986,2020-12-29,185.55,193.81
11448,603986,2020-12-30,191.88,195.22


In [92]:
#获取股票代码列表
stocks = set(data['Stkcd'].values)

In [95]:
#筛选数据，剔除有交易异常的股票
codes = []
for i in stocks:
    if data[data['Stkcd']==i].shape[0]==243:
        codes.append(i)

In [99]:
data1 = data[data['Stkcd'].isin(codes)].copy() #copy后不改变原数据

In [100]:
data1['return'] = (data1['Clsprc']-data1['Opnprc'])/data1['Opnprc']

In [123]:
#转化为收益率矩阵
stocks = [str(i) for i in codes]
ret = pd.DataFrame()
for i in codes:
    ret[str(i)] = data1[data1['Stkcd'] == i]['return'].values

In [301]:
ret

,601088,601601,601857,600196,600837,600585,600588,603259,603288,601628,...,600809,601066,603501,600050,600436,600309,600438,601336,600570,601211
0,0.008681,0.009870,0.001706,0.010086,-0.007619,0.009042,0.058412,-0.009406,0.002320,-0.004841,...,-0.001237,0.001920,0.039819,0.020270,-0.007001,-0.010796,0.063910,0.015984,0.007994,-0.006915
1,0.001079,0.005584,0.013629,0.001848,0.005758,-0.023571,-0.000330,-0.026077,-0.028479,-0.003135,...,-0.004091,0.002258,-0.035519,-0.003300,-0.004566,0.018605,0.012694,-0.001176,-0.001516,0.009110
2,0.011309,-0.022785,0.033167,-0.006696,0.016077,-0.012775,0.016584,-0.016580,-0.010504,-0.002591,...,-0.015143,0.020132,-0.001129,0.006689,0.003232,-0.026455,0.057828,-0.012392,0.021711,0.019293
3,0.006417,0.003594,-0.009724,0.006737,-0.004425,0.006777,0.000324,0.011041,0.020900,0.006877,...,0.003811,0.028405,-0.015000,-0.001658,0.010189,-0.007609,0.000659,0.007357,0.019363,-0.002619
4,-0.025491,-0.005168,-0.004792,-0.016455,-0.021879,-0.005849,0.004916,-0.000458,0.001604,-0.017842,...,0.013873,-0.019872,0.009609,-0.010033,0.002097,-0.010398,-0.046735,-0.013917,0.001486,-0.009043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.023269,-0.011449,0.004819,-0.029791,0.007968,-0.004865,0.025000,0.047225,0.041451,0.004552,...,0.026677,0.000750,-0.034622,0.008989,0.012146,0.040023,0.066211,-0.012153,0.027432,0.011144
239,-0.003759,0.024011,-0.007194,-0.049571,-0.007911,-0.018768,-0.020673,-0.019182,-0.011959,-0.001335,...,0.066369,-0.007828,0.017682,0.004464,-0.004024,-0.031797,0.010973,0.015631,-0.015769,-0.011034
240,-0.024324,0.000828,-0.002415,-0.019392,-0.011111,0.019406,0.049325,-0.001931,0.003865,0.008558,...,0.004520,0.001519,-0.014667,-0.004435,-0.027850,0.010975,-0.076481,0.004561,-0.002331,0.000000
241,0.003898,-0.001103,0.002427,0.010537,0.004827,0.002734,-0.008702,0.014772,0.026013,-0.002136,...,0.008760,0.027813,0.033545,-0.008909,0.045492,0.024023,0.014706,-0.002105,0.082888,0.004689


In [608]:
T = ret.shape[0]
ret_mean = dict(ret.mean())
exp_ret = 0.004

In [609]:
#model
m1 = grb.Model('MAD')
#var
x = m1.addVars(stocks,name='invest')
y = m1.addVars(T,name='y')
#obj
m1.setObjective(y.sum()/T,sense=grb.GRB.MINIMIZE)
#cons
m1.addConstrs(y[i]+grb.quicksum((ret[s][i]-ret_mean[s])*x[s] for s in stocks)>=0 for i in range(T))
m1.addConstrs(y[i]-grb.quicksum((ret[s][i]-ret_mean[s])*x[s] for s in stocks)>=0 for i in range(T))

m1.addConstr(x.sum()<=1,name='budget')
m1.addConstr(x.prod(ret_mean)>=exp_ret,name='exp_ret')
#optimize
m1.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 488 rows, 286 columns and 21470 nonzeros
Model fingerprint: 0xef686a55
Coefficient statistics:
  Matrix range     [3e-06, 1e+00]
  Objective range  [4e-03, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 1e+00]
Presolve removed 6 rows and 3 columns
Presolve time: 0.05s
Presolved: 482 rows, 283 columns, 21206 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.560000e-01   0.000000e+00      0s
     273    7.8532898e-03   0.000000e+00   0.000000e+00      0s

Solved in 273 iterations and 0.10 seconds
Optimal objective  7.853289810e-03


In [610]:
#print
if m1.status == grb.GRB.OPTIMAL:
    for v in m1.getVars()[:43]:
        if v.x >= 1e-6:
            print(v.varName,v.x)
    stock_invset = [s for s in stocks if x[s].x >= 1e-6]
    print(stock_invset)

invest[603288] 0.08154157667298463
invest[601888] 0.0055237812842113005
invest[601633] 0.10465911843609806
invest[601899] 0.09365769106978268
invest[601398] 0.025376532539178355
invest[600809] 0.2747090349710795
invest[603501] 0.03460554043558649
invest[601336] 0.014087760863355381
['603288', '601888', '601633', '601899', '601398', '600809', '603501', '601336']


In [611]:
profolio1 = 0
sm = 0
for i in stock_invset:
    profolio1 += x[i].x*ret[i].values
    sm += x[i].x

In [612]:
#利用总投资比例对夏普比率进行修正
sharp1 = (profolio1.mean() - 0.002)/(profolio1.std()*sm)
print(sharp1)

0.2998463710808427


In [613]:
#model
m2 = grb.Model('SMAD')

#var
x = m2.addVars(stocks,name='invest')
y = m2.addVars(T,name='y')

#obj
m2.setObjective(y.sum()/T,sense=grb.GRB.MINIMIZE)

#cons
m2.addConstrs(y[i]+grb.quicksum((ret[s][i]-ret_mean[s])*x[s] for s in stocks)>=0 for i in range(T))
m2.addConstr(x.sum()<=1,name='budget')
m2.addConstr(x.prod(ret_mean)>=exp_ret,name='exp_ret')

#optimize
m2.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 245 rows, 286 columns and 10778 nonzeros
Model fingerprint: 0x4c46ce1f
Coefficient statistics:
  Matrix range     [3e-06, 1e+00]
  Objective range  [4e-03, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 1e+00]
Presolve removed 3 rows and 3 columns
Presolve time: 0.06s
Presolved: 242 rows, 283 columns, 10646 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.560000e-01   0.000000e+00      0s
     157    3.9266449e-03   0.000000e+00   0.000000e+00      0s

Solved in 157 iterations and 0.08 seconds
Optimal objective  3.926644905e-03


In [614]:
#print
if m2.status == grb.GRB.OPTIMAL:
    for v in m2.getVars()[:43]:
        if v.x >= 1e-6:
            print(v.varName,v.x)
    stock_invset = [s for s in stocks if x[s].x >= 1e-6]
    print(stock_invset)

invest[603288] 0.08154157667298477
invest[601888] 0.005523781284211225
invest[601633] 0.10465911843609794
invest[601899] 0.09365769106978254
invest[601398] 0.025376532539178383
invest[600809] 0.2747090349710796
invest[603501] 0.03460554043558646
invest[601336] 0.014087760863355541
['603288', '601888', '601633', '601899', '601398', '600809', '603501', '601336']


In [615]:
profolio2 = 0
sm = 0
for i in stock_invset:
    profolio2 += x[i].x*ret[i].values
    sm += x[i].x

In [616]:
sharp2 = (profolio2.mean() - 0.002)/(profolio2.std()*sm)
print(sharp2)

0.2998463710808427


In [669]:
alpha = 0.02
aT = int(alpha*T)

In [670]:
#model
m3 = grb.Model('cvar')
#var
x = m3.addVars(stocks,name='invest')
s = m3.addVar(name='s')
p = m3.addVars(T,name='p')
#obj
m3.setObjective(aT*s+p.sum(),sense=grb.GRB.MINIMIZE)
#cons
m3.addConstrs(s+p[i] >= (-1/aT)*x.prod(dict(ret.iloc[i,:])) for i in range(T))
m3.addConstr(x.sum()<=1,name='budget')
m3.addConstr(x.prod(ret_mean)>=exp_ret,name='exp_ret')
#optimize
m3.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 245 rows, 287 columns and 10705 nonzeros
Model fingerprint: 0x6e0fa2d1
Coefficient statistics:
  Matrix range     [5e-06, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 1e+00]
Presolve time: 0.02s
Presolved: 245 rows, 287 columns, 10705 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.560000e-01   0.000000e+00      0s
       5    2.7044785e-02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.04 seconds
Optimal objective  2.704478514e-02


In [671]:
#print
if m3.status == grb.GRB.OPTIMAL:
    for v in m3.getVars()[:43]:
        if v.x >= 1e-6:
            print(v.varName,v.x)
    stock_invset = [s for s in stocks if x[s].x >= 1e-6]
    print(stock_invset)

invest[603288] 0.26633166333537517
invest[601633] 0.33416814567516034
invest[600809] 0.057516416864699334
['603288', '601633', '600809']


In [672]:
profolio3 = 0
sm = 0
for i in stock_invset:
    profolio3 += x[i].x*ret[i].values
    sm += x[i].x

In [673]:
sharp3 = (profolio3.mean() - 0.002)/(profolio3.std()*sm)
print(sharp3)

0.2296578748993454


In [762]:
alpha = 0.05
delta = 30
aT = int(alpha*(T-delta))

In [763]:
dd = pd.DataFrame()
for i in stocks:
    dd[i] = [max(ret[i][j-delta:j])-ret[i][j] for j in range(delta,T)]

In [764]:
#model
m4 = grb.Model('CDaR')

#vars
x = m4.addVars(stocks,name='invest')
y = m4.addVar(name='y')
z = m4.addVars(T-delta,name='z')

#obj
m4.setObjective(y+(1/aT)*z.sum(),sense=grb.GRB.MINIMIZE)

#cons
m4.addConstrs(z[i] >= (-1/aT)*x.prod(dict(dd.iloc[i,:]))-y for i in range(T-delta))
m4.addConstr(x.sum()<=1,name='budget')
m4.addConstr(x.prod(ret_mean)>=exp_ret,name='exp_ret')

#optimize
m4.optimize()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 215 rows, 257 columns and 9670 nonzeros
Model fingerprint: 0xd7be82ad
Coefficient statistics:
  Matrix range     [2e-06, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 1e+00]
Presolve removed 98 rows and 98 columns
Presolve time: 0.03s
Presolved: 117 rows, 159 columns, 5260 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.560000e-01   0.000000e+00      0s
       4    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.05 seconds
Optimal objective  0.000000000e+00


In [765]:
#print
if m4.status == grb.GRB.OPTIMAL:
    for v in m4.getVars()[:43]:
        if v.x >= 1e-6:
            print(v.varName,v.x)
    stock_invset = [s for s in stocks if x[s].x >= 1e-6]
    print(stock_invset)

invest[600588] 0.17451238548225778
invest[600809] 0.40608885755741897
invest[601066] 0.04962968632921911
['600588', '600809', '601066']


In [766]:
profolio4 = 0
sm = 0
for i in stock_invset:
    profolio4 += x[i].x*ret[i].values
    sm += x[i].x

In [767]:
sharp4 = (profolio4.mean() - 0.002)/(profolio4.std()*sm) 
print(sharp4)

0.23831148108107414
